# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


In [40]:
class Matrix:
    def __init__(self, n=None, m=None, values=None):
        if values is not None:  
            if not all(len(row) == len(values[0]) for row in values):
                raise ValueError("All rows must have the same number of columns.")
            self.data = values
            self.n = len(values)
            self.m = len(values[0])
        elif n is not None and m is not None:  
            self.n = n
            self.m = m
            self.data = [[0 for _ in range(m)] for _ in range(n)]
        else:
            raise ValueError("Invalid initialization arguments.")
            
    def __getitem__(self, index):
        if isinstance(index, tuple):
            i, j = index
            return self.data[i][j]
        elif isinstance(index, slice):  
            return self.data[index]
        else:
            return self.data[index]

    def __setitem__(self, index, value):
        if isinstance(index, tuple):
            i, j = index
            self.data[i][j] = value
        else:
            raise ValueError("Invalid index format")
            
    def assign(self, other):
        if isinstance(other, Matrix): 
            if self.n == other.n and self.m == other.m:
                self.data = [row[:] for row in other.data]  
            else:
                raise ValueError("Matrix dimensions must match for assignment.")
        elif isinstance(other, list):  
            if len(other) == self.n and all(len(row) == self.m for row in other):
                self.data = [row[:] for row in other]
            else:
                raise ValueError("List of lists must match the matrix dimensions.")
        else:
            raise TypeError("Can only assign from another matrix or a list of lists.")
            
            
        # Question 2    
    def shape(self):
        return (self.n, self.m)

    def transpose(self):
        transposed_data = [[self.data[j][i] for j in range(self.n)] for i in range(self.m)]
        return Matrix(values=transposed_data)

    def row(self, n):
        if n >= self.n:
            raise ValueError("Row index out of range.")
        return Matrix(values=[self.data[n]])

    def column(self, n):
        if n >= self.m:
            raise ValueError("Column index out of range.")
        return Matrix(values=[[self.data[i][n]] for i in range(self.n)])

    def to_list(self):
        return self.data

    def block(self, n_0, n_1, m_0, m_1):
        if not (0 <= n_0 <= n_1 <= self.n and 0 <= m_0 <= m_1 <= self.m):
            raise ValueError("Invalid block indices.")
        block_data = [row[m_0:m_1] for row in self.data[n_0:n_1]]
        return Matrix(values=block_data)
    
    
         # Question 4

    def scalarmul(self, c):
        return Matrix(values=[[self.data[i][j] * c for j in range(self.m)] for i in range(self.n)])

    def add(self, N):
        if self.n != N.n or self.m != N.m:
            raise ValueError("Matrices must have the same dimensions to be added.")
        return Matrix(values=[[self.data[i][j] + N.data[i][j] for j in range(self.m)] for i in range(self.n)])

    def sub(self, N):
        if self.n != N.n or self.m != N.m:
            raise ValueError("Matrices must have the same dimensions to be subtracted.")
        return Matrix(values=[[self.data[i][j] - N.data[i][j] for j in range(self.m)] for i in range(self.n)])

    def mat_mult(self, N):
        if self.m != N.n:
            raise ValueError("Matrix multiplication not possible: M's columns must equal N's rows.")
        result = [[sum(self.data[i][k] * N.data[k][j] for k in range(self.m)) for j in range(N.m)] for i in range(self.n)]
        return Matrix(values=result)

    def element_mult(self, N):
        if self.n != N.n or self.m != N.m:
            raise ValueError("Matrices must have the same dimensions for element-wise multiplication.")
        return Matrix(values=[[self.data[i][j] * N.data[i][j] for j in range(self.m)] for i in range(self.n)])

    def equals(self, N):
        if self.n != N.n or self.m != N.m:
            return False
        return all(self.data[i][j] == N.data[i][j] for i in range(self.n) for j in range(self.m))

    # Question 5

    def __rmul__(self, c):
        return self.scalarmul(c)

    def __mul__(self, c):
        if isinstance(c, Matrix):
            return self.mat_mult(c)
        else:
            return self.scalarmul(c)

    def __add__(self, N):
        return self.add(N)

    def __sub__(self, N):
        return self.sub(N)

    def __eq__(self, N):
        return self.equals(N)

    def __copy__(self):
        return Matrix(values=[row[:] for row in self.data])
        

In [52]:
#Test
M1 = Matrix(3, 3)
print(M1.data) 

M2 = Matrix(values=[[7, 3, 7], [2, 9, 6], [8, 0, 6]])
print(M2.data)  

print(M2[1, 2]) 

M2[1, 2] = 8
print(M2[1, 2])  

M3 = Matrix(3, 3)
M3.assign(M2)
print(M3.data)  

M3.assign([[5, 1, 2], [9, 8, 3], [6, 0, 3]])
print(M3.data) 

[[0, 0, 0], [0, 0, 0], [0, 0, 0]]
[[7, 3, 7], [2, 9, 6], [8, 0, 6]]
6
8
[[7, 3, 7], [2, 9, 8], [8, 0, 6]]
[[5, 1, 2], [9, 8, 3], [6, 0, 3]]


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

In [42]:
#Test
M = Matrix(values=[[1, 2, 3], [4, 5, 6], [7, 8, 9]])

print("Shape of the matrix:", M.shape()) 

transpose_M = M.transpose()
print("Transpose of the matrix:", transpose_M.to_list()) 

row_1 = M.row(1)
print("Row 1 of the matrix:", row_1.to_list())  

column_2 = M.column(2)
print("Column 2 of the matrix:", column_2.to_list()) 

matrix_as_list = M.to_list()
print("Matrix as a list of lists:", matrix_as_list)  

block_M = M.block(1, 3, 0, 2)  
print("Block of the matrix (rows 1-2, columns 0-1):", block_M.to_list()) 

Shape of the matrix: (3, 3)
Transpose of the matrix: [[1, 4, 7], [2, 5, 8], [3, 6, 9]]
Row 1 of the matrix: [[4, 5, 6]]
Column 2 of the matrix: [[3], [6], [9]]
Matrix as a list of lists: [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
Block of the matrix (rows 1-2, columns 0-1): [[4, 5], [7, 8]]


3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

In [43]:
def constant(n, m, c):
    return [[float(c) for _ in range(m)] for _ in range(n)]

def zeros(n, m):
    return constant(n, m, 0)

def ones(n, m):
    return constant(n, m, 1)

def eye(n):
    return [[1.0 if i == j else 0.0 for j in range(n)] for i in range(n)]
# Test 
print("Constant 3x3 matrix with value 5.0:", constant(3, 3, 5))

print("3x3 zeros matrix:", zeros(3, 3))

print("3x3 ones matrix:", ones(3, 3))

print("3x3 identity matrix:", eye(3))

Constant 3x3 matrix with value 5.0: [[5.0, 5.0, 5.0], [5.0, 5.0, 5.0], [5.0, 5.0, 5.0]]
3x3 zeros matrix: [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]
3x3 ones matrix: [[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0, 1.0]]
3x3 identity matrix: [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]


4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

In [44]:
# Test matrices
M1 = Matrix(values=[[1, 2], [3, 4]])
M2 = Matrix(values=[[5, 6], [7, 8]])

M_scalar = M1.scalarmul(2)
print("Scalar multiplication of M1 by 2:", M_scalar.to_list())  

M_add = M1.add(M2)
print("Matrix addition (M1 + M2):", M_add.to_list())  

M_sub = M1.sub(M2)
print("Matrix subtraction (M1 - M2):", M_sub.to_list())  

M_mult = M1.mat_mult(M2)
print("Matrix multiplication (M1 * M2):", M_mult.to_list())  

M_elem_mult = M1.element_mult(M2)
print("Element-wise multiplication (M1 * M2):", M_elem_mult.to_list())  

print("M1 equals M2:", M1.equals(M2))  
M3 = Matrix(values=[[1, 2], [3, 4]])
print("M1 equals M3:", M1.equals(M3))  

Scalar multiplication of M1 by 2: [[2, 4], [6, 8]]
Matrix addition (M1 + M2): [[6, 8], [10, 12]]
Matrix subtraction (M1 - M2): [[-4, -4], [-4, -4]]
Matrix multiplication (M1 * M2): [[19, 22], [43, 50]]
Element-wise multiplication (M1 * M2): [[5, 12], [21, 32]]
M1 equals M2: False
M1 equals M3: True


5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


In [46]:
# Test matrices
M1 = Matrix(values=[[1, 2], [3, 4]])
M2 = Matrix(values=[[5, 6], [7, 8]])

M_scalar1 = 2 * M1
M_scalar2 = M1 * 2
print("2 * M1:", M_scalar1.to_list())  
print("M1 * 2:", M_scalar2.to_list()) 

M_add = M1 + M2
print("M1 + M2:", M_add.to_list())  

M_sub = M1 - M2
print("M1 - M2:", M_sub.to_list())  

M_mult = M1 * M2
print("M1 * M2 (matrix multiplication):", M_mult.to_list())  

print("M1 == M2:", M1 == M2)  
M3 = Matrix(values=[[1, 2], [3, 4]])
print("M1 == M3:", M1 == M3)  

M4 = M1.__copy__()
M4[0, 0] = 10
print("M1 after copying to M4 and modifying M4:", M1.to_list())  
print("M4 after modification:", M4.to_list()) 

2 * M1: [[2, 4], [6, 8]]
M1 * 2: [[2, 4], [6, 8]]
M1 + M2: [[6, 8], [10, 12]]
M1 - M2: [[-4, -4], [-4, -4]]
M1 * M2 (matrix multiplication): [[19, 22], [43, 50]]
M1 == M2: False
M1 == M3: True
M1 after copying to M4 and modifying M4: [[1, 2], [3, 4]]
M4 after modification: [[10, 2], [3, 4]]


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [47]:
A = Matrix(values=[[1, 2], [3, 4]])
B = Matrix(values=[[5, 6], [7, 8]])
C = Matrix(values=[[2, 0], [1, 2]])

I = Matrix(values=[[1, 0], [0, 1]])

AB = A * B
ABC_1 = AB * C  
BC = B * C
ABC_2 = A * BC  

print("Associativity: (AB)C == A(BC):", ABC_1.to_list() == ABC_2.to_list())  

B_plus_C = B + C
LHS = A * B_plus_C  
AB = A * B
AC = A * C
RHS = AB + AC 

print("Distributive property: A(B + C) == AB + AC:", LHS.to_list() == RHS.to_list())  

BA = B * A
print("Non-commutative: AB != BA:", AB.to_list() != BA.to_list())  

AI = A * I
print("Identity property: AI == A:", AI.to_list() == A.to_list())  

Associativity: (AB)C == A(BC): True
Distributive property: A(B + C) == AB + AC: True
Non-commutative: AB != BA: True
Identity property: AI == A: True
